In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, \
    classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
import os

enable_one_hot = False
enable_scaling = False

# Load Data
# os.chdir("/home/pier/Machine_Learning/KE5206NN/diabetes_svm")
os.chdir("/Users/pierlim/PycharmProjects/KE5206NN/diabetes_svm")
dfs = pd.read_excel("data/diabetic_data.xlsx", sheet_name=None)
df = dfs['in']
df = df.iloc[:, 2:]
print(df.shape)

# Data Exploration
corr = df.corr()
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()
df.describe()

df_numeric = df.select_dtypes(include=[np.number])

# drop missing values
df = df.replace('?', np.nan)
df = df.replace('Unknown/Invalid', np.nan)
print(df.columns[df.isnull().any()])
df.isnull().sum()
# df = df.dropna() # dangerous! dropped until left 1k plus rows

print(df.shape)
df = df.drop(columns=df.columns[df.nunique() == 1])  # drop columns which only have 1 category

to_num = ['time_in_hospital', 'num_lab_procedures', 'num_procedures',
          'num_medications', 'number_outpatient', 'number_emergency',
          'number_inpatient', 'number_diagnoses']

to_cat_codes = list(set(df.columns) - set(to_num))

X_features = list(to_num)
for c in to_cat_codes:
    df[c] = df[c].astype('category')
    df[c + '_cat'] = df[c].cat.codes
    X_features += [c + '_cat']

X_features.remove('readmitted_cat')

for n in to_num:
    df[n] = df[n].astype('int')

df[to_num].dtypes

X_train, X_test, y_train, y_test = train_test_split(
    df[X_features], df['readmitted_cat'], random_state=0)

clf = LinearSVC(max_iter=5000, dual=False, C=10, random_state=42).fit(X_train, y_train.values.ravel())
print('training accuracy: {:.2f}'.format(clf.score(X_train, y_train)))
print('test accuracy: {:.2f}'.format(clf.score(X_test, y_test)))


(101766, 48)


<Figure size 640x480 with 2 Axes>

Index(['race', 'gender', 'weight', 'payer_code', 'medical_specialty'], dtype='object')
(101766, 48)
training accuracy: 0.57
test accuracy: 0.57
